In [1]:
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')
import os
import scanpy as sc
import pandas as pd

from scmamba2 import logger
from scmamba2.utils.metrics import biology_conservation, omics_mixing
from scmamba2.utils.evaluation import ARI

In [2]:
concat_emb = sc.read_h5ad("../results/PBMC10kbatchsize256projection_dim32/concat.h5ad")
concat_emb

AnnData object with n_obs × n_vars = 19262 × 32
    obs: 'rna:cell_type', 'atac:cell_type', 'cell_type', 'leiden', 'modality'
    uns: 'cell_type_colors', 'leiden', 'leiden_colors', 'modality_colors', 'neighbors', 'umap'
    obsm: 'X_umap'
    obsp: 'connectivities', 'distances'

In [3]:
logger.info("Calculating omics mixing metrics...")
omics_mixing_metrics = omics_mixing(
    concat_emb, concat_emb.obs['cell_type'].values, concat_emb.obs['modality'].values
)
omics_mixing_metrics

scMamba - INFO - Calculating omics mixing metrics...


/home/zyuan/.conda/envs/mamba/lib/python3.10/site-packages/anndata/_core/anndata.py:402: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/zyuan/code/scMamba2/scmamba2/utils/evaluation.py:298: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  counts = pd.value_counts(c)


{'omics entropy mixing score': 0.9897355481833218,
 'Seurat alignment score (omics)': 0.9829785005174084,
 'Graph connectivity': 0.6124171572338926,
 'ASW_omics': 0.7810567617416382}

In [4]:
logger.info("Calculating biology conservation metrics...")
biology_conservation_metrics, best_res= biology_conservation(
    concat_emb, concat_emb.obs['cell_type'].values
)
print(best_res)
biology_conservation_metrics

scMamba - INFO - Calculating biology conservation metrics...
1.0


{'ARI': 0.7798572644798041,
 'NMI': 0.7777392690075673,
 'Mean average precision': 0.742387808962666,
 'ASW_celltype': 0.6086784154176712}

In [3]:
max_res = 0
max_ARI = 0
for res in range(40, 60):
    res = res/100.0
    sc.tl.leiden(concat_emb, resolution=res, flavor="igraph", n_iterations=2)
    ARI_score = ARI(concat_emb.obs["cell_type"].values, concat_emb.obs["leiden"].values)
if ARI_score > max_ARI:
    max_res = res
    max_ARI = ARI_score
print(max_res)
print(max_ARI)

0.59
0.7887402943898991


In [ ]:
metrics = {}
metrics_classified = {}
logger.info("Calculating biology conservation metrics...")
biology_conservation_metrics, best_res= biology_conservation(
    concat_emb, concat_emb.obs['cell_type'].values
)
logger.info("Calculating omics mixing metrics...")
omics_mixing_metrics = omics_mixing(
    concat_emb, concat_emb.obs['cell_type'].values, concat_emb.obs['modality'].values
)
metrics_classified['biology conservation'] = biology_conservation_metrics
metrics_classified['omics mixing'] = omics_mixing_metrics

metrics.update(biology_conservation_metrics)
metrics.update(omics_mixing_metrics)

metrics['epcohs'] = 100
metrics['best resolutioin'] = best_res
# metrics['mean F1 silhouette'] = mean_F1_silhouette(
#     concate_embeds.X, 
#     cell_type=concate_embeds.obs['cell_type'].values,
#     omics=concate_embeds.obs['modality'].values,
#     device_id=args.device,
#     chunk_size=100000
# )
print(metrics)

if not os.path.exists(f'{out_dir}/metrics.csv'):
    metrics_df = pd.DataFrame([metrics])
else:
    metrics_df = pd.read_csv(
        f'{out_dir}/metrics.csv' 
    )
    new_metrics_df = pd.DataFrame([metrics])
    metrics_df = pd.concat([metrics_df, new_metrics_df], ignore_index=True)
metrics_df.to_csv(f'{out_dir}/metrics.csv', index=False)